In [22]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, precision_score

In [6]:
path_csv_file = "C:/Users/husey/PycharmProjects/pythonProject12/IntToNLP_GRP_9/latest_hukums_with_classes_csv_file1.csv"

df = pd.read_csv(path_csv_file)
df.head()

,Unnamed: 0,Unnamed: 0.1,ictihat,Suç,YeniSuclar
0,0,0,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",hakaret,Şerefe karşı suçlar
1,1,1,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",tehdit,Hürriyete karşı suçlar
2,2,2,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",yaralama,Vücut dokunulmazlığına karşı suçlar
3,3,3,HÜKÜMLER : Beraat Yerel Mahkemece bozma ...,tehdit,Hürriyete karşı suçlar
4,4,4,HÜKÜMLER : Beraat Yerel Mahkemece bozma ...,hakaret,Şerefe karşı suçlar


In [10]:
label_map = {}
count = 0
for label in df.YeniSuclar.unique():
    label_map[label] = count
    count += 1
df['NUM_LABEL'] = df.YeniSuclar.map(label_map)
#print(label_map)
df.YeniSuclar.value_counts()

Mal varlığına karşı suçlar                                    3137
Hürriyete karşı suçlar                                        2883
Vücut dokunulmazlığına karşı suçlar                           2684
others                                                        2364
Kamu güvenine karşı suçlar                                    1993
Şerefe karşı suçlar                                           1317
Kamunun sağlığına karşı suçlar                                1216
Kamu idaresinin güvenilirliğine ve işleyişine karşı suçlar     837
Anayasal düzene ve bu düzenin işleyişine karşı suçlar          779
Cinsel dokunulmazlığa karşı suçlar                             607
Adliyeye karşı suçlar                                          401
Hayata karşı suçlar                                            254
Genel tehlike yaratan suçlar                                   249
Çevreye karşı suçlar                                           171
Ekonomi, sanayi ve ticarete ilişkin suçlar                    

In [11]:
texts = df.ictihat
label_nums = df.NUM_LABEL

X_train, X_test, y_train, y_test = train_test_split(texts, label_nums, test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(15424,)
(3856,)
(15424,)
(3856,)


In [12]:
def clean_text(text):
    alphabetic_only = [word for word in text.split() if word.isalpha()]
    lower_case_only = [word.lower() for word in alphabetic_only]
    stopwords_tr = set(stopwords.words("turkish"))
    return [word for word in lower_case_only if word not in stopwords_tr]


In [13]:
vect = CountVectorizer(analyzer='word', tokenizer=clean_text)
X_train_dtm = vect.fit_transform(X_train.astype("U"))
X_train_dtm

<15424x23422 sparse matrix of type '<class 'numpy.int64'>'
	with 1543725 stored elements in Compressed Sparse Row format>

In [14]:
X_test_dtm = vect.transform(X_test.astype("U"))
X_test_dtm

<3856x23422 sparse matrix of type '<class 'numpy.int64'>'
	with 384498 stored elements in Compressed Sparse Row format>

In [15]:
"""
vect2 = TfidfTransformer()
X_train_tfidf = vect2.fit_transform(X_train_dtm.astype("U"))
X_train_tfidf
"""

'\nvect2 = TfidfTransformer()\nX_train_tfidf = vect2.fit_transform(X_train_dtm.astype("U"))\nX_train_tfidf\n'

In [16]:
"""
X_test_tfidf = vect2.transform(X_test_dtm.astype("U"))
X_test_tfidf
"""

'\nX_test_tfidf = vect2.transform(X_test_dtm.astype("U"))\nX_test_tfidf\n'

In [24]:
model = LogisticRegression(solver='lbfgs', max_iter=3000)
model.fit(X_train_dtm, y_train)

LogisticRegression(max_iter=3000)

In [26]:
y_pred_class = model.predict(X_test_dtm)
print('********************************LogisticRegression********************************')
print(metrics.accuracy_score(y_test, y_pred_class))
print('********************************Confusion Matrix********************************')
print(confusion_matrix(y_test, y_pred_class))
print('********************************Precision score macro********************************')
print(precision_score(y_test, y_pred_class, average='macro', zero_division=1))
print('********************************Precision score micro********************************')
print(precision_score(y_test, y_pred_class, average='micro', zero_division=1))
print('********************************Precision score weighted********************************')
print(precision_score(y_test, y_pred_class, average='weighted', zero_division=1))
print('********************************Precision score null********************************')
print(precision_score(y_test, y_pred_class, average=None, zero_division=1))
print('********************************Classification report********************************')
print(classification_report(y_test, y_pred_class, zero_division=1))

********************************LogisticRegression********************************
0.5842842323651453
********************************Confusion Matrix********************************
[[ 72 104  32  10   4  27   0   1   0   0   3   0   0   0   1   0   1   0
    0]
 [ 87 244  69  18  94   7   0  10   8   0  28   2   0   4   0   0   0   0
    1]
 [ 40  59 380  19  14   5   0  22   3   2   7   0   0   0   0   0   0   1
    2]
 [  8  60  23 172  76  16  16  16   7  17  16   2   1   0  38   0   2   0
    2]
 [  3  55  14  58 414  12   0   0   3   2   2   0   3   0  35   0   0   0
    0]
 [ 19   9   7  22  21  84   0   2   3   0   0   2   0   0   1   0   0   0
    1]
 [  0   0   0  11   0   0 228   0   1   0   0   0   0   0   9   0   0   0
    0]
 [  2  27   8  15   2   2   0  90   3   1   0   1   1   1   0   1   0   0
    0]
 [  1  10   0  11   6   0   0  16  53   0   0   0   0   2   4   0   0   0
    1]
 [  1   1   3  19   0   0   0   5   0  28   0   0   0   0   0   0   0   0
    0]
 [  4  